In [ ]:
#필요한 패키지와 라이브러리를 가져온다

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

#정보를 추출하는 함수를 생성한다. 이때 별점 정보가 없으면 0으로, 가격 정보가 없으면 50000으로 설정한다.

def extract_info(tags):
  info = []

  for item in range(len(tags)):
    name = tags[item].select_one('div.name').get_text(strip = True)
    stars = tags[item].select_one('em.rating')
    stars = '0' if stars == None else stars.get_text()
    price = tags[item].select_one('strong.price_value')
    price = '50,000' if price == None else price.get_text()
    price = price.replace(',','')

    info.append({'상품명' : name, '판매가격' : price, '별점' : stars })

  return info

#쿠팡 카테고리의 모든 페이지에서 상품명과 판매가격, 별점을 가져와서 df라는 DataFrame에 저장한다.

url = 'https://www.coupang.com/np/categories/194282'
headers = {'User-Agent' : 'Mozilla/5.0'}

result = []

for page in range(1,18):
  params = {'page' : page}
  response = requests.get(url, params = params , headers = headers)

  if response.status_code == 200:
    soup = BeautifulSoup(response.text , 'html.parser')
    tags = soup.select('dd.descriptions')
    res = extract_info(tags)
    result.extend(res)
  else:
    print("Error")

df = pd.DataFrame(result)

#별점을 판매가격으로 나누고 10000을 scailing한 값을 가성비라고 정의하고 DataFrame에 추가한다.

df['판매가격'] = df['판매가격'].astype(int)
df['별점'] = df['별점'].astype(float)
df['가성비'] = df['별점']/df['판매가격']*10000

#가성비가 2 이상인 상품들과 2 미만인 상품들의 수량을 별점에 따라 분포하도록 히스토그램을 plot 한다.
#여기서 얻어낼 수 있는 결론은 가성비 즉, 가격 대비 만족도가 높은 제품은 대부분 높은 별점 쪽에 치우쳐 있고,
#그렇지 않은 제품은 낮은 별점에서도 일정량이 보인다는 것이다.

cost_high_mask = df['가성비'] >= 2
cost_high = df[cost_high_mask]
cost_low_mask = df['가성비'] < 2
cost_low = df[cost_low_mask]

import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
%matplotlib inline

plt.rc('font', family = 'Malgun Gothic')
plt.hist(cost_high.별점 , label = '가성비 2 이상', width = 0.3)
plt.hist(cost_low.별점 , label = '가성비 2 미만', alpha = 0.5 , width = 0.3)
plt.legend(loc = 'best')
plt.xlabel('별점')
plt.ylabel('수량')
